In [4]:

!pip install -qr https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt  



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install  opencv-python mediapipe matplotlib

  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/f3/0b/92b29c7d829ce7a69575200bee7d0d0843fe52e84c227a51d17a2223b851/mediapipe-0.10.18-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for attrs>=19.1.0 from https://files.pythonhosted.org/packages/6a/21/5b6702a7f963e95456c0de2d495f67bf5fd62840ac655dc451586d23d39a/attrs-24.2.0-py3-none-any.whl.metadata
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Obtaining dependency information for jax from https://files.pythonhosted.org/packages/62/20/6c57c50c0ccc645fea1895950f1e5cd02f961ee44b3ffe83617fa46b0c1d/jax-0.4.35-py3-none-any.whl.metadata
  Using cached jax-0.4.35-py3-none-any.whl.metadata (22 kB)
  Obtaining dependency information for jaxlib from https://files.pythonhosted.org/packages/97/05/093b3c511837ba514f0b97581f7b21e1bb79768b8b9c29013a406b00d484/jaxlib-0.4.35-cp312-cp312-win_amd64.whl.metadata
  Using cached jaxlib-0.4.35-cp312-cp312-win_amd64.whl.metada


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import matplotlib.pyplot as plt

from PIL import Image
import cv2

import mediapipe as mp
from mediapipe.python.solutions import pose as mp_pose

#if you are using colab
from google.colab.patches import cv2_imshow

# PyTorch Hub
import torch

In [8]:
# Model
yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

#since we are only intrested in detecting person
yolo_model.classes=[0]

Using cache found in C:\Users\Yuurei/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-30 Python-3.12.7 torch-2.5.0+cpu CPU

100%|██████████| 14.1M/14.1M [00:45<00:00, 323kB/s] 

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [9]:
mp_drawing = mp.solutions.drawing_utils
mp_pose =mp.solutions.pose

In [15]:
video_path =r"C:\Users\Yuurei\PycharmProjects\poseYOLOV8MultiHuman\snapvideo.io--DJ Snake, LISA - SG remix choreography.mp4"

In [16]:
#get the dimension of the video
cap = cv2.VideoCapture(video_path)
while cap.isOpened():
    ret, frame = cap.read()
    h, w, _ = frame.shape
    size = (w, h)
    print(size)
    break

(1280, 720)


In [17]:
cap = cv2.VideoCapture(video_path)
#for webacam cv2.VideoCapture(NUM) NUM -> 0,1,2 for primary and secondary webcams..
#For saving the video file as output.avi
out = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 20, size)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
      break
    # Recolor Feed from RGB to BGR
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #making image writeable to false improves prediction
    image.flags.writeable = False
    result = yolo_model(image)
    # Recolor image back to BGR for rendering
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    # print(result.xyxy)  # img1 predictions (tensor)
    #This array will contain crops of images incase we need it
    img_list =[]
    #we need some extra margin bounding box for human crops to be properly detected
    MARGIN=10

    for (xmin, ymin, xmax,   ymax,  confidence,  clas) in result.xyxy[0].tolist():
      with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
        #Media pose prediction ,we are
        results = pose.process(image[int(ymin)+MARGIN:int(ymax)+MARGIN,int(xmin)+MARGIN:int(xmax)+MARGIN:])

        #Draw landmarks on image, if this thing is confusing please consider going through numpy array slicing
        mp_drawing.draw_landmarks(image[int(ymin)+MARGIN:int(ymax)+MARGIN,int(xmin)+MARGIN:int(xmax)+MARGIN:], results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                              )
        img_list.append(image[int(ymin):int(ymax),int(xmin):int(xmax):])
    # cv2_imshow(image)
    # writing in the video file
    out.write(image)
    ## Code to quit the video incase you are using the webcam
    # cv2.imshow('Activity recognition', image)
    # if cv2.waitKey(10) & 0xFF == ord('q'):
    #     break
cap.release()
out.release()
cv2.destroyAllWindows()


C:\Users\Yuurei/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Yuurei/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Yuurei/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Yuurei/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Yuurei/.cache\torch\hub\ultralytics_yolov5_master\m

In [14]:
import cv2
import time

cap = cv2.VideoCapture(video_path)

# Đối với webcam, sử dụng cv2.VideoCapture(NUM), trong đó NUM là 0, 1, 2... cho các webcam chính và phụ.

# Tạo đối tượng ghi video để lưu video dưới dạng output.avi
out = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 20, size)

# Biến để tính FPS
prev_time = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Tính thời gian hiện tại
    current_time = time.time()
    fps = 1 / (current_time - prev_time)
    prev_time = current_time
    
    # Hiển thị FPS trên khung hình
    cv2.putText(frame, f"FPS: {int(fps)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Chuyển đổi màu từ BGR sang RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Thiết lập cờ ghi để không cho phép chỉnh sửa hình ảnh, giúp cải thiện hiệu suất dự đoán
    image.flags.writeable = False

    # Thực hiện dự đoán với mô hình YOLO
    result = yolo_model(image)

    # Chuyển đổi lại màu từ RGB sang BGR để hiển thị
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Mảng này sẽ chứa các ảnh cắt nếu cần
    img_list = []

    # Thêm một khoảng cách nhỏ để đảm bảo hộp giới hạn (bounding box) đủ lớn để nhận diện người
    MARGIN = 10

    for (xmin, ymin, xmax, ymax, confidence, clas) in result.xyxy[0].tolist():
        with mp_pose.Pose(min_detection_confidence=0.3, min_tracking_confidence=0.3) as pose:
            # Dự đoán tư thế với Mediapipe
            results = pose.process(image[int(ymin) + MARGIN:int(ymax) + MARGIN, int(xmin) + MARGIN:int(xmax) + MARGIN])

            # Vẽ các điểm mốc (landmarks) trên ảnh, nếu không rõ về slicing mảng numpy, bạn nên tìm hiểu thêm
            mp_drawing.draw_landmarks(
                image[int(ymin) + MARGIN:int(ymax) + MARGIN, int(xmin) + MARGIN:int(xmax) + MARGIN],
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
            
            # Thêm phần ảnh đã cắt vào mảng img_list
            img_list.append(image[int(ymin):int(ymax), int(xmin):int(xmax)])

    # Ghi khung hình vào tệp video
    out.write(image)

    # Mã lệnh để thoát video nếu sử dụng webcam
    # cv2.imshow('Nhận diện hoạt động', image)
    # if cv2.waitKey(10) & 0xFF == ord('q'):
    #     break

# Giải phóng các tài nguyên
cap.release()
out.release()
cv2.destroyAllWindows()


C:\Users\Yuurei/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Yuurei/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Yuurei/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Yuurei/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users\Yuurei/.cache\torch\hub\ultralytics_yolov5_master\m